# Company Database Example with PostgreSQL and psycopg2

This notebook demonstrates how to create and interact with a PostgreSQL database using Python and the `psycopg2` library. It covers database creation, table setup, data insertion, and common queries, with explanations for each step.

In [ ]:
import psycopg2
import psycopg2.errors
import sys

# Database connection parameters
# Edit these as needed for your environment
DB_NAME = 'company'
USER = 'postgres'
PASSWORD = '1234'
HOST = 'localhost'

**Explanation:**

This code block imports the required libraries (`psycopg2` for PostgreSQL access and `sys` for system exit). It also sets up the database connection parameters such as database name, user, password, and host. The database name is now set to `company`. You should update these parameters to match your local PostgreSQL setup.

## 1. Create the Database (if it does not exist)

We first connect to the default `postgres` database and attempt to create the `company` database. If it already exists, we continue without error.

In [ ]:
try:
    tmp_conn = psycopg2.connect(dbname='postgres', user=USER, password=PASSWORD, host=HOST)
    tmp_conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    with tmp_conn.cursor() as tmp_cur:
        try:
            tmp_cur.execute(f"CREATE DATABASE {DB_NAME};")
            print(f"Database '{DB_NAME}' created.")
        except psycopg2.errors.DuplicateDatabase:
            print(f"Database '{DB_NAME}' already exists.")
    tmp_conn.close()
except Exception as e:
    print(f"Error creating database: {e}")
    sys.exit(1)

**Explanation:**

This code block attempts to create the `company` database by connecting to the default `postgres` database. If the database already exists, it catches the error and prints a message. If any other error occurs, it prints the error and exits the notebook. This ensures the target database is available for the rest of the notebook.

## 2. Connect to the Database

Now we connect to the `company` database to perform all further operations.

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, user=USER, password=PASSWORD, host=HOST)
cur = conn.cursor()
print('Connected to database.')

**Explanation:**

This code block connects to the `company` database using the parameters defined earlier. It creates a connection and a cursor object, which are used to execute SQL commands in subsequent cells.

## 3. Create Tables

We create the `departments` and `employees` tables if they do not already exist.

In [ ]:
cur.execute('''
CREATE TABLE IF NOT EXISTS departments (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL
);
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    department_id INTEGER REFERENCES departments(id),
    position VARCHAR(50),
    salary NUMERIC
);
''')
print('Tables created (if not already present).')

**Explanation:**

This code block creates two tables: `departments` and `employees`. The `departments` table stores department names, and the `employees` table stores employee details, including a foreign key reference to the department. The `IF NOT EXISTS` clause ensures the tables are only created if they do not already exist.

## 4. Insert Sample Data

Insert sample departments and employees. If the data already exists, it will not be duplicated.

In [ ]:
cur.execute('''
INSERT INTO departments (name) VALUES
('Engineering'),
('HR'),
('Sales'),
('Marketing'),
('Finance')
ON CONFLICT DO NOTHING;
''')

cur.execute('''
INSERT INTO employees (name, department_id, position, salary) VALUES
('Alice', 1, 'Engineer', 80000),
('Bob', 1, 'Senior Engineer', 95000),
('Carol', 2, 'HR Manager', 70000),
('Dave', 3, 'Sales Rep', 60000),
('Eve', 4, 'Marketing Lead', 72000),
('Frank', 1, 'DevOps Engineer', 85000),
('Grace', 5, 'Accountant', 65000),
('Heidi', 4, 'Content Strategist', 68000),
('Ivan', 3, 'Sales Manager', 78000),
('Judy', 5, 'Financial Analyst', 70000)
ON CONFLICT DO NOTHING;
''')
print('Sample data inserted.')

**Explanation:**

This code block inserts sample data into the `departments` and `employees` tables. The `ON CONFLICT DO NOTHING` clause prevents duplicate entries if the data already exists. This provides a consistent dataset for running queries and examples in the notebook.

## 5. Query Data

Run several example queries to explore the data and relationships.

### a. Query employees with their department names

In [ ]:
# Query employees with their department names
cur.execute('''
SELECT e.name AS employee, d.name AS department, e.position, e.salary
FROM employees e
JOIN departments d ON e.department_id = d.id;
''')
print('Employees with department:')
for row in cur.fetchall():
    print(row)

**Explanation:**

This code block queries all employees, joining the `employees` and `departments` tables to show each employee's name, department, position, and salary. The results are printed row by row.

### b. List all employees in the 'Engineering' department

In [ ]:
# List all employees in the 'Engineering' department
cur.execute('''
SELECT name FROM employees
WHERE department_id = (SELECT id FROM departments WHERE name = 'Engineering');
''')
print("\nEmployees in Engineering:")
for row in cur.fetchall():
    print(row)

**Explanation:**

This code block lists all employees who work in the 'Engineering' department. It uses a subquery to find the department ID for 'Engineering' and then selects all employees with that department ID.

### c. Count employees per department

In [ ]:
# Count employees per department
cur.execute('''
SELECT d.name AS department, COUNT(e.id) AS num_employees
FROM departments d
LEFT JOIN employees e ON d.id = e.department_id
GROUP BY d.name;
''')
print("\nEmployee count per department:")
for row in cur.fetchall():
    print(row)

**Explanation:**

This code block counts the number of employees in each department. It uses a LEFT JOIN to include departments with zero employees and groups the results by department name.

### d. Find the highest paid employee in each department

In [ ]:
# Find the highest paid employee in each department
cur.execute('''
SELECT d.name AS department, e.name AS employee, MAX(e.salary) AS max_salary
FROM departments d
JOIN employees e ON d.id = e.department_id
GROUP BY d.name, e.name
ORDER BY d.name, max_salary DESC;
''')
print("\nHighest paid employee in each department:")
for row in cur.fetchall():
    print(row)

**Explanation:**

This code block finds the highest paid employee in each department. It joins the two tables, groups by department and employee, and orders the results by department and salary.

### e. List employees with salary above 75,000

In [ ]:
# List all employees in the 'Engineering' department
cur.execute('''
SELECT name FROM employees
WHERE department_id = (SELECT id FROM departments WHERE name = 'Engineering');
''')
print("\nEmployees in Engineering:")
for row in cur.fetchall():
    print(row)

**Explanation:**

This code block lists all employees whose salary is greater than 75,000. It simply selects the name and salary columns for those employees.

## 6. Update and Delete Data

Show how to update and delete records in the database.

In [ ]:
# Change an employee's department
cur.execute('''
UPDATE employees SET department_id = 2 WHERE name = 'Frank';
''')
conn.commit()
print("Frank's department updated to HR.")

# Delete all employees in the Sales department
cur.execute('''
DELETE FROM employees WHERE department_id = (SELECT id FROM departments WHERE name = 'Sales');
''')
conn.commit()
print("All employees in Sales department deleted.")

**Explanation:**

This code block demonstrates how to update and delete data. It first updates Frank's department to HR, then deletes all employees in the Sales department. The `conn.commit()` calls ensure the changes are saved to the database.

## 7. Delete Database & Close the Connection

Always close the cursor and connection when finished.

In [ ]:
cur.close()
conn.close()
print('Database connection closed.')

**Explanation:**

This code block closes the cursor and the database connection, releasing all resources. This is good practice to avoid leaving open connections to the database.